# Data Extraction (Text-to-KG)

In [21]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j loguru pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 56.6 MB/s eta 0:00:00


In [1]:
!git clone https://github.com/Ahmad-Wali-i170106/kgrag-poc.git

Cloning into 'kgrag-poc'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (301/301), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 301 (delta 187), reused 207 (delta 95), pack-reused 0 (from 0)
Receiving objects: 100% (301/301), 5.01 MiB | 10.24 MiB/s, done.
Resolving deltas: 100% (187/187), done.


In [13]:
%cd kgrag-poc

/content/kgrag-poc


In [14]:
!pwd

/content/kgrag-poc


In [24]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Uncomment the below to use LangSmith. Not required.
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

··········


In [19]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI


In [43]:
import os
import time
from typing import List, Dict, Any
# from dotenv import load_dotenv
from langchain.globals import set_llm_cache
# from langchain_community.cache import InMemoryCache
from langchain_core.documents.base import Document


'''
You need the following environment variables:

    1. NEO4J_URL (Default="bolt://localhost:7687"),
    2. NEO4J_USERNAME (Default="neo4j"),
    3. NEO4J_PASSWORD (Default=None),
    4. NEO4J_DATABASE (Default="neo4j")
    (The above are for establishing a Neo4j Connection and can be passed as arguments as well)
    5. MODELS_CACHE_FOLDER (Default=None):-
        I'm using SentenceTransformer model "all-MiniLM-v6-L2" for creating embeddings to calculate vector similarity.
        This is just the path to the folder to use as the cache folder
    6. GOOGLE_API_KEY (For ChatGoogleGenerativeAI) or any other API key for env variable if you want to use some other LLM
    7. TESSDATA_PREFIX (Might need it for using PyTesseract)
'''
# load_dotenv('../.env')


os.environ["NEO4J_URL"] = "neo4j+s://8387f2bf.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "ROqhMguw-WnlJYhRc8jPvUi1v6EqmtTwrcCEGwc0eCA"

from kgrag.data_extraction import Text2KG
# from kgrag.parse_pdf import PDFParserMarkdown, OCREngine
from kgrag.md_chunks import docs_to_md, chunk_md

# from langchain_google_genai import ChatGoogleGenerativeAI

# set_llm_cache(InMemoryCache()) # Set LLM Caching (Optional)

# Replace with llm you want to use
# llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash-002", temperature=0)

llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")

llm_transformer = LLMGraphTransformer(llm=llm)

In [31]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://8387f2bf.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "ROqhMguw-WnlJYhRc8jPvUi1v6EqmtTwrcCEGwc0eCA"

graph = Neo4jGraph()

In [26]:
!pwd

/content/kgrag-poc


In [27]:
# filepath = input("Please enter filepath of the file you want to process: ")
# filepath = "../SampleDocs/Leadership-Etsko-Schuitema.pdf" # ...or any other file you want to use
filepath = "monetary-policy-report-august-2024.pdf" # ...or any other file you want to use
# filepath = "/media/wali/D_Drive/Documents/Books/C++_Programming_Program_Design_Including_Data_Structure_D.S.Malik_5th_DS.pdf"
# filepath = "/media/wali/D_Drive/Documents/Books/C-How-to-Program-7th-Edition.pdf"

In [44]:

# Can also accept neo4j_url, neo4j_username, neo4j_password & neo4j_database arguments
# if the above mentioned environment variables have not been set
text2kg = Text2KG(
    llm=llm,
    emb_model=None, # By Default, SentenceTransformer's all-MiniLM-v6-L2 is used or use any other embedding model
    link_nodes=True,
    node_vector_similarity_threshold=0.90,
    subject="Kosmos Finance Agreement", #filepath.split('/')[-1].split('.')[0].replace('_',' ').replace('-',' '), # Subject can be anything or nothing - filename works well for most cases
    verbose=True
)

AttributeError: 'ChatOpenAI' object has no attribute 'model'

In [ ]:
# '''
# Parse or read the PDF file
# Can use any parser as long as some conditions are observed in the output:
#     1. Must be in langchain_core.documents.base.Document format
#     2. Must contain the following keys in doc.metadata:
#         1. page
#         2. filename/filepath or source
# '''

# pages =  list(range(37, 45)) #list(range(68,70)) # None

# parser = PDFParserMarkdown(
#     pdf_path=filepath,
#     pages=pages, # Can pass a list of pages to read, useful for debugging
#     ocr_engine=OCREngine.PYTESSERACT, # 3 OCR Options: PYTESSERACT, LLM, RAPIDOCR (LLM is most accurate)
# )

# doc_dicts: List[Dict[str, Any]] = parser.process_pdf_document()

# docs: List[Document] = [
#     Document(
#         page_content=doc['text'],
#         metadata={**doc['page_metadata'], **doc['doc_metadata']}
#     )
#     for doc in doc_dicts
# ]

In [ ]:
mds = docs_to_md(filepath)
all_docs = []
for md in mds:
    chunks = chunk_md(text=md)

    docs: List[Document] = [
        Document(
            page_content=chunk['text'],
            metadata={
                "chunk_id": int(chunk_id),
                "start": chunk['start'],
                "end": chunk['end'],
                "filepath": filepath,
                "source": f"{filepath.split('/')[-1]} Chunk {chunk_id}: {chunk['start']} - {chunk['end']}"
            }
        )
        for chunk_id, chunk in chunks.items()
    ]

    all_docs.extend(docs)

docs = all_docs

Processing ../SampleDocs/kosmos.pdf...
[                                        ] (0/43[                                        ] (  1/43[                                        ] (  2/43[                                        ] (  3/43[                                        ] (  4/43[                                        ] (  5/43[                                        ] (  6/43[                                        ] (  7/43[                                        ] (  8/43[                                        ] (  9/43[                                        ] ( 10/430[=                                       ] ( 11/4[=                                       ] ( 12/430[=                                       ] ( 13/4[=                                       ] ( 14/430[=                                       ] ( 15/4[=                                       ] ( 16/430[=                                       ] ( 17/4[=                                       ] ( 18/430[=          

In [ ]:
print(docs[-1].page_content)

-----|Amount provided by a person/persons to KEO or Obligors made available for the purpose of meeting projected liabilities unrelated to the Borrowing Base Assets that the Facility Agent is satisfied will be available (including amounts available to be drawn under RCF)|Col2|scheduled and default interest, fees, costs and expenses related to the Revolving Credit Facility and HY Notes otherwise referred to as Scheduled KEL Debt Payments over the next 12 months|Col4|
|---|---|---|---|
|||any other material committed liability for the next 12 months period including any guarantee, indemnity or other contingent liability, which could be reasonably be expected to entail a cash outflow for the next 12 month period||
|||||
|TOTAL KEO AND ITS SUBSIDIARIES||TOTAL KEO AND ITS SUBSIDIARIES||


288 -----


In [ ]:
start_time = time.process_time()
text2kg.process_documents(docs, use_existing_node_types=False)
end_time = time.process_time()
print(f"Total Time: {end_time - start_time}")

2024-10-16 14:06:04.286 | INFO     | kgrag.data_extraction:docs2nodes_and_rels:513 - Doc # 1
Nodes:
[Node(id='Kosmos Energy Finance International', type='Company', properties=[Property(key='role', value='Borrower')], aliases=[], definition=''), Node(id='Kosmos Energy Operating', type='Company', properties=[], aliases=[], definition=''), Node(id='Kosmos Energy International', type='Company', properties=[], aliases=[], definition=''), Node(id='Kosmos Energy Development', type='Company', properties=[], aliases=[], definition=''), Node(id='Kosmos Energy Ghana Hc', type='Company', properties=[], aliases=[], definition=''), Node(id='Kosmos Energy Equatorial Guinea', type='Company', properties=[], aliases=[], definition=''), Node(id='Kosmos Energy Ghana Investments', type='Company', properties=[], aliases=[], definition=''), Node(id='Kosmos Energy Ghana Holdings Limited', type='Company', properties=[], aliases=[], definition=''), Node(id='Kosmos Equatorial Guinea Inc', type='Company', propert

# KG Search / Retrieval

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")

from langchain_google_genai import ChatGoogleGenerativeAI

from kgrag.kg_search import KGSearch


llm = ChatGoogleGenerativeAI(model="models/gemini-1.0-pro", temperature=0)



In [ ]:

kg_search = KGSearch(
    ent_llm=llm,
    cypher_llm=llm,
    cypher_examples_json="examples.json",
    fulltext_search_top_k=6,
    vector_search_top_k=6,
    vector_search_min_score=0.7
)


In [ ]:
'''
OR get lists of strings separately using kg_search.retrieve - retrieve_as_string is only a wrapper for this function

rels, docs, gen_cypher_results = kg_search.retrieve(
    query,
    nresults=30,
    use_fulltext_search=True,
    use_vector_search=True,
    generate_cypher=False,
    return_chunk_ids=True
)
rels: All the triples that the entities/nodes in the query are involved in
     Empty list is returned if `use_fulltext_search=False` & `use_vector_search=False`
docs: All the documents that contain any entities mentioned in the query
     Empty list is returned if `use_fulltext_search=False` & `use_vector_search=False`
     If return_chunk_ids=True, returns chunk_ids as set during text2kg conversion / data insertion process
gen_cypher_results: List of string/JSON results from the generated cypher
     Empty list is returned if `generate_cypher=False`

If no entities are found using vector or fulltext search, then a cypher is generated regardless of the value of `generate_cypher`
And results are returned in the gen_cypher_results
WARNING: Using `generate_cypher` is unreliable and error-prone. Needs more work and more examples from 'examples.json'
'''

In [ ]:

query = "Describe the differences between web 3.0 and web 2.0." #input("Enter your query: ")

docs_string = kg_search.retrieve_as_string( #rels, docs, gen_res
    query,
    nresults=500,
    use_fulltext_search=True, # Extract all entities (using ent_llm) in the input query and search using fulltext search
    use_vector_search=True, # Search for all entities/nodes in the query using vector search
    generate_cypher=False # Use LLM (cypher_llm) to generate cypher - Uses examples from `cypher_examples_json` for guidance
)
# print(len(rels))

In [ ]:
q = f"""Answer the given as best as you can. Use the given search results to assist you with answering the query
Query: {query}

Context:
{docs_string}

Answer:"""
ans = llm.invoke(q)

In [ ]:
print(ans.content)

**Web 3.0**

Web 3.0 refers to the next movement in web development—one that realizes the full potential of the web. The Internet in its current state is a giant conglomeration of single websites with loose connections. Web 3.0 will resolve this by moving toward the Semantic Web—or the “web of meaning”—in which the web becomes a giant database meaningfully searchable by computers.

**Web 2.0**

Web 2.0 has no single definition but can be explained through a series of Internet trends, one being the empowerment of the user. Companies such as eBay, Facebook and Twitter are built almost entirely on community-generated content. Web 2.0 takes advantage of collective intelligence, the idea that collaboration will result in intelligent ideas. For example, wikis, such as the encyclopedia Wikipedia, allow users access to edit content. Tagging, or labeling content, is another key part of the collaborative theme of Web 2.0, which can be seen in sites such as Flickr, a photo-sharing site, and del.i

In [ ]:
print(docs_string)

Nodes Relations:-
(Q113420780: Technology:Node {'wiki_description': 'scientific article published on 11 June 2022', 'alias': ['Web 3.0'], 'definition': 'The next movement in web development—one that realizes the full potential of the web', 'wiki_type': 'scholarly article', 'url': 'www.wikidata.org/wiki/Q113420780', 'labels': ['Web 3.0: The Future of Web']})-[:USES]->(Q134471: Technology:Node {'wiki_description': 'group of interrelated Web development techniques', 'alias': ['Ajax'], 'definition': 'A set of web development techniques using many web technologies on the client-side to create asynchronous web applications', 'wiki_type': 'acronym', 'url': 'www.wikidata.org/wiki/Q134471', 'labels': ['AJAX', 'Asynchronous JavaScript and XML']})
(Q113420780: Technology:Node {'wiki_description': 'scientific article published on 11 June 2022', 'alias': ['Web 3.0'], 'definition': 'The next movement in web development—one that realizes the full potential of the web', 'wiki_type': 'scholarly article